### 소개

이 파일은 이전까지 진행된 EDA를 기반으로 column name들을 rename 하고 새로운 파생변수를 생성해서 모델에서 학습할 수 있도록 train_df.csv, test_df.csv 파일로 save 되는 과정이 담겨져 있습니다

### Library Import

In [1]:
import os
from typing import List, Dict, Tuple
from tqdm import tqdm
import numpy as np
import pandas as pd

### Data Load

In [3]:
# 파일 호출
data_path: str = "data/raw"
train_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv")).assign(_type="train") # train 에는 _type = train
test_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")).assign(_type="test") # test 에는 _type = test
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")) # ID, target 열만 가진 데이터 미리 호출
df: pd.DataFrame = pd.concat([train_df, test_df], axis=0)

In [4]:
# HOURLY_ 로 시작하는 .csv 파일 이름을 file_names 에 할딩
file_names: List[str] = [
    f for f in os.listdir(data_path) if f.startswith("HOURLY_") and f.endswith(".csv")
]

# 파일명 : 데이터프레임으로 딕셔너리 형태로 저장
file_dict: Dict[str, pd.DataFrame] = {
    f.replace(".csv", ""): pd.read_csv(os.path.join(data_path, f)) for f in file_names
}

for _file_name, _df in tqdm(file_dict.items()):
    # 열 이름 중복 방지를 위해 {_file_name.lower()}_{col.lower()}로 변경, datetime 열을 ID로 변경
    _rename_rule = {
        col: f"{_file_name.lower()}_{col.lower()}" if col != "datetime" else "ID"
        for col in _df.columns
    }
    _df = _df.rename(_rename_rule, axis=1)
    df = df.merge(_df, on="ID", how="left")


100%|█████████████████████████████████████████| 107/107 [00:01<00:00, 76.49it/s]


### Feature engineering

가격 지표는 test_df에는 포함 되어 있지 않지만 가격을 회귀로 예측하여 예측한 값을 입력피쳐로 활용하여 target을 예측하여 볼 수 있고 파생지표 또한 활용할 수 있는 방안이 있어 df에 포함시킵니다.

In [5]:
close_price = df['hourly_market-data_price-ohlcv_all_exchange_spot_btc_usd_close']

In [6]:
# 모델에 사용할 컬럼, 컬럼의 rename rule을 미리 할당함
cols_dict: Dict[str, str] = {
    "ID": "ID",
    "target": "target",
    "_type": "_type",
    "hourly_market-data_coinbase-premium-index_coinbase_premium_gap": "coinbase_premium_gap",
    "hourly_market-data_funding-rates_all_exchange_funding_rates": "funding_rates",
    "hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations": "long_liquidations",
    "hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations": "short_liquidations",
    "hourly_market-data_open-interest_all_exchange_all_symbol_open_interest": "open_interest",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_ratio": "buy_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_sell_ratio": "buy_sell_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_volume": "buy_volume",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_ratio": "sell_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_volume": "sell_volume",
    "hourly_network-data_addresses-count_addresses_count_active": "active_count",
    "hourly_network-data_addresses-count_addresses_count_receiver": "receiver_count",
    "hourly_network-data_addresses-count_addresses_count_sender": "sender_count",
    'hourly_network-data_block-interval_block_interval':'block_interval',
    'hourly_network-data_block-count_block_count':'block_count',
    'hourly_network-data_block-bytes_block_bytes':'block_bytes',
    'hourly_network-data_blockreward_blockreward':'blockreward',
    'hourly_network-data_transactions-count_transactions_count_total': 'transaction_count',
    'hourly_network-data_tokens-transferred_tokens_transferred_total': 'token_transferred',
    'hourly_network-data_tokens-transferred_tokens_transferred_mean':
    'token_transferred_mean',
    'hourly_network-data_tokens-transferred_tokens_transferred_median':
    'token_transferred_median',
    'hourly_network-data_hashrate_hashrate':
    'hashrate',
    'hourly_network-data_difficulty_difficulty':
    'difficulty',
    'hourly_network-data_fees-transaction_fees_transaction_mean':
    'fees_transaction',
    'hourly_network-data_fees_fees_total':
    'fees',
    'hourly_network-data_velocity_velocity_supply_total':
    'velocity_supply',
    'hourly_network-data_utxo-count_utxo_count':
    'utxo_count',
    'hourly_network-data_supply_supply_total':
    'supply_total',
    'hourly_network-data_supply_supply_new':
    'supply_new',
    'hourly_network-data_fees_fees_block_mean':
    'fees_block_mean',
    'hourly_network-data_fees-transaction_fees_transaction_median':
    'fees_transaction_median',



}

df = df[cols_dict.keys()].rename(cols_dict, axis=1)
df.shape


(11552, 34)

In [7]:
# 새로 만든 컬럼들을 담을 리스트 conti_cols
conti_cols = []

# cols_dict에서 ID, target, _type과 같은 변수들을 제외하고 conti_cols에 추가
exclude_keys = ['ID', 'target', '_type']

# 조건에 맞게 연속형 변수로 간주할 수 있는 컬럼들만 추가
for key, value in cols_dict.items():
    if key not in exclude_keys:
        conti_cols.append(value)

In [8]:
df['close_price'] = close_price

In [9]:
# eda 에서 파악한 차이와 차이의 음수, 양수 여부를 새로운 피쳐로 생성
df = df.assign(
    liquidation_diff=df["long_liquidations"] - df["short_liquidations"],
    volume_diff=df["buy_volume"] - df["sell_volume"],
    liquidation_diffg=np.sign(df["long_liquidations"] - df["short_liquidations"]),
    volume_diffg=np.sign(df["buy_volume"] - df["sell_volume"]),
    buy_sell_volume_ratio=df["buy_volume"] / (df["sell_volume"] + 1),
)


In [10]:
# 새로 추가한 칼럼들 conti_cols에 추가
new_cols = ["liquidation_diff", "volume_diff", "buy_sell_volume_ratio"]

# 이미 만들어진 conti_cols에 추가
conti_cols.extend(new_cols)

#### Market 파생변수

마켓 관련된 파생변수를 추가합니다

In [11]:
# 0 나오는거 방지
eps = 1e-8
df['coinbase_premium_gap'] = df['coinbase_premium_gap'] + eps


# 1. volatility 지수 | 변동성 지표
df['premium_gap_volatility'] = df['coinbase_premium_gap'].rolling(window=6).std()

# 2. liquidation(long and short) 대비 open interest 비율
df['long_liquidation_to_open_interest_ratio'] = df['long_liquidations'] / df['open_interest']
df['short_liquidation_to_open_interest_ratio'] = df['short_liquidations'] / df['open_interest']

# 3. Funding rate MACD
df['funding_rate_ema_short'] = df['funding_rates'].ewm(span=12).mean()
df['funding_rate_ema_long'] = df['funding_rates'].ewm(span=24).mean()
df['funding_rate_macd'] = df['funding_rate_ema_short'] - df['funding_rate_ema_long']

# 4. open interest 불린져 밴드 계산
oi_ma6 = df['open_interest'].rolling(window=6).mean()  # 6기간 이동 평균
oi_std6 = df['open_interest'].rolling(window=6).std()   # 6기간 이동 표준 편차

df['oi_upper_band'] = oi_ma6 + oi_std6 * 2
df['oi_lower_band'] = oi_ma6 - oi_std6 * 2

# 5. funding_rates 사용하여 RSI 계산
delta = df['funding_rates'].diff()
up = delta.clip(lower=0)
down = -1 * delta.clip(upper=0)
roll_up = up.rolling(window=6).mean()
roll_down = down.rolling(window=6).mean()
rs = roll_up / (roll_down + 1e-9)
df['funding_rates_rsi'] = 100 - (100 / (1 + rs))

# 6. 이동 지수 표준 편차
df['oi_ewm_volatility'] = df['open_interest'].ewm(span=3, adjust=False).std()

# 7. 시장 총 거래량
df['total_volume'] = df['buy_volume'] + df['sell_volume']

# 8. buying selling volume 차이 MA
df['taker_volume_oscillator'] = df['buy_volume'].rolling(window=5).mean() - df['sell_volume'].rolling(window=5).mean()

# 9. Market sentiment index
df['market_sentiment'] = (df['buy_ratio'] - df['sell_ratio']) * df['open_interest']

In [12]:
# 새로 생성된 컬럼들 리스트
new_market_cols = [
    'premium_gap_volatility',
    'long_liquidation_to_open_interest_ratio',
    'short_liquidation_to_open_interest_ratio',
    'funding_rate_ema_short',
    'funding_rate_ema_long',
    'funding_rate_macd',
    'oi_upper_band',
    'oi_lower_band',
    'funding_rates_rsi',
    'oi_ewm_volatility',
    'total_volume',
    'taker_volume_oscillator',
    'market_sentiment'
]

# conti_cols에 새로운 피처 추가
conti_cols.extend(new_market_cols)

#### Network 파생변수

네트워크 관련된 파생변수를 추가합니다

In [13]:
# 0 방지
df['supply_new'] = df['supply_new']+eps

# 1. UTXO growth rate
df['utxo_growth_rate'] = df['utxo_count'].pct_change() * 100

# 2. Average UTXO per Transaction
df['avg_utxo_per_transaction'] = df['utxo_count'] / df['transaction_count']

# 3. Velocity Change Rate
df['velocity_change_rate'] = df['velocity_supply'].pct_change() * 100

# 4. Velocity to UTXO Ratio
df['velocity_to_utxo_ratio'] = df['velocity_supply'] / df['utxo_count']

# 5. Velocity to Supply Ratio
df['velocity_to_supply_ratio'] = df['velocity_supply'] / df['supply_total']

# 6. Active address growth
df['active_address_growth_rate'] = df['active_count'].pct_change() * 100

# 7. Sender-Receiver Ratio
df['sender_receiver_ratio'] = df['sender_count'] / df['receiver_count']

# 8. Address Distribution Index
df['address_distribution_index'] = (df['sender_count'] + df['receiver_count']) / df['active_count']

# 9. Block Interval Change Rate
df['block_interval_change_rate'] = df['block_interval'].pct_change() * 100

# 10. Block Utilization
df['block_utilization'] = df['block_bytes'] / df['block_count']

# 11. Block Reward to Difficulty Ratio
df['block_reward_to_difficulty_ratio'] = df['blockreward'] / df['difficulty']

# 12. Fee to Transaction Ratio
df['fee_to_transaction_ratio'] = df['fees_transaction'] / df['transaction_count']

# 13. Fees Growth Rate
df['fees_growth_rate'] = df['fees'].pct_change() * 100

# 14. Mining Cost Efficiency
df['mining_cost_efficiency'] = df['fees_block_mean'] / df['difficulty']

# 15. Transaction per Supply Ratio
df['transaction_per_supply_ratio'] = df['transaction_count'] / df['supply_total']

# 16. Velocity to Supply Ratio
df['velocity_to_supply_ratio'] = df['velocity_supply'] / df['supply_total']

# 17. Supply Change Rate
df['supply_change_rate'] = (df['supply_total'].diff() / df['supply_total'].shift(1)) * 100

# 18. UTXO Growth Rate
df['utxo_growth_rate'] = df['utxo_count'].pct_change() * 100

# 19. Token Transfer Efficiency
df['token_transfer_efficiency'] = df['token_transferred'] / df['transaction_count']

# 20. Velocity to UTXO Ratio
df['velocity_to_utxo_ratio'] = df['velocity_supply'] / df['utxo_count']

# 21. network activity
df['network_activity_index'] = df['active_count'] * (df['receiver_count'] + df['sender_count'])


In [14]:
# 새로 추가된 컬럼들 리스트
new_network_cols = [
    'utxo_growth_rate',
    'avg_utxo_per_transaction',
    'velocity_change_rate',
    'velocity_to_utxo_ratio',
    'velocity_to_supply_ratio',
    'active_address_growth_rate',
    'sender_receiver_ratio',
    'address_distribution_index',
    'block_interval_change_rate',
    'block_utilization',
    'block_reward_to_difficulty_ratio',
    'fee_to_transaction_ratio',
    'fees_growth_rate',
    'mining_cost_efficiency',
    'transaction_per_supply_ratio',
    'supply_change_rate',
    'token_transfer_efficiency',
    'network_activity_index'
]

# conti_cols에 새로운 피처 추가
conti_cols.extend(new_network_cols)


#### 가격 관련 파생변수

test_df 에는 가격 데이타가 없습니다. 하지만, 가격 예측을 어느 정도 맞게 할 수 있다면 비트코인 시장의 시그널을 잘 보여주는 파생변수를 가격을 이용해 만들 수 있습니다.

가격


In [15]:
# stock to flow ratio 계산 이후 SF reversion 계산
df['stock_to_flow_ratio'] = df['supply_total']/df['supply_new']
df['SF_reversion'] = df['close_price']/df['stock_to_flow_ratio']
df['SF_reversion']

# market cap 계산
df['market_cap'] = df['close_price'] * df['supply_total']

# NVT ratio
df['NVT_Ratio'] = df['market_cap']/df['transaction_count']

# NVT ratio의 평균과 표준편차를 이용해 불린저 밴드 계산
window = 72 # 임의적으로 72 시간 설정

NVT_ma = df['NVT_Ratio'].rolling(window=window).mean()
NVT_std = df['NVT_Ratio'].rolling(window=window).std()

# 상단 밴드 및 하단 밴드 설정 (Bollinger Bands)
df['NVT_Upper_Band'] = NVT_ma + 2 * NVT_std
df['NVT_Lower_Band'] = NVT_ma - 2 * NVT_std



In [16]:
# 새로 만든 피처들 리스트
new_price_cols = [
    'stock_to_flow_ratio',
    'SF_reversion',
    'market_cap',
    'NVT_Ratio',
    'NVT_Upper_Band',
    'NVT_Lower_Band'
]

# conti_cols에 새로운 피처 추가
conti_cols.extend(new_price_cols)

# 중복 방지
conti_cols = list(set(conti_cols))
len(conti_cols)

71

#### Diff, MA, Shift 피쳐 추가

기본적으로 시장의 흐름에 따라 가격의 변동이 달라질 수 있으므로 흐름을 반영할 수 있는 지표들이 중요합니다. 이를 위해 지금까지 continuous 한 column 들로 모아놓은 conti_cols에서 difference, difference의 difference (지표가 변하는 속력과 가속도), shift, ma를 계산해서 지표로 넣을 필요가 있습니다.

In [17]:
def diff_feature(
    df: pd.DataFrame,
    conti_cols: List[str],
) -> Tuple[List[pd.Series], List[pd.Series]]:
    """
    연속형 변수의 diff와 diff의 diff를 계산하여 리스트로 반환
    Args:
        df (pd.DataFrame): 데이터프레임
        conti_cols (List[str]): 연속형 변수 컬럼 리스트
    Return:
        (List[pd.Series], List[pd.Series]): diff_list, diff_diff_list
    """
    # diff 계산
    diff_list = [
        df[conti_col].diff().rename(f"{conti_col}_diff")
        for conti_col in conti_cols
    ]

    # diff의 diff 계산
    diff_diff_list = [
        df[conti_col].diff().diff().rename(f"{conti_col}_diff_diff")
        for conti_col in conti_cols
    ]

    return diff_list, diff_diff_list

# diff와 diff의 diff 계산
diff_list, diff_diff_list = diff_feature(df=df, conti_cols=conti_cols)


In [18]:
def shift_feature(
    df: pd.DataFrame,
    conti_cols: List[str],
    intervals: List[int],
) -> List[pd.Series]:
    """
    연속형 변수의 shift feature 생성
    Args:
        df (pd.DataFrame)
        conti_cols (List[str]): continuous colnames
        intervals (List[int]): shifted intervals
    Return:
        List[pd.Series]
    """
    df_shift_dict = [
        df[conti_col].shift(interval).rename(f"{conti_col}_{interval}")
        for conti_col in conti_cols
        for interval in intervals
    ]
    return df_shift_dict

# 최대 12시간의 shift 피쳐를 계산
shift_list = shift_feature(
    df=df, conti_cols=conti_cols, intervals=[_ for _ in range(1, 12)]
)

In [19]:
from typing import List
import pandas as pd

def ma_feature(
    df: pd.DataFrame,
    conti_cols: List[str],
    window_sizes: List[int]
) -> List[pd.Series]:
    """
    연속형 변수의 이동평균(MA)을 여러 윈도우 크기로 계산하여 리스트로 반환
    Args:
        df (pd.DataFrame): 데이터프레임
        conti_cols (List[str]): 연속형 변수 컬럼 리스트
        window_sizes (List[int]): MA를 계산할 여러 윈도우 크기 리스트
    Return:
        List[pd.Series]: ma_list
    """
    # 여러 윈도우 크기에 대해 이동평균(MA) 계산
    ma_list = [
        df[conti_col].rolling(window=window_size).mean().rename(f"{conti_col}_ma_{window_size}")
        for conti_col in conti_cols
        for window_size in window_sizes
    ]

    return ma_list

# 여러 윈도우 크기로 이동평균(MA) 계산 3, 6, 12 시간 적용
ma_list = ma_feature(df=df, conti_cols=conti_cols, window_sizes=[3, 6, 12])


In [20]:
# 새로 생성된 리스트들을 데이터프레임으로 변환
diff_df = pd.concat(diff_list, axis=1)
diff_diff_df = pd.concat(diff_diff_list, axis=1)
shift_df = pd.concat(shift_list, axis=1)
ma_df = pd.concat(ma_list, axis=1)

# 원본 df에 새로운 피처들을 concat
df = pd.concat([df, diff_df, diff_diff_df, shift_df, ma_df], axis=1)


# 타겟 변수를 제외한 변수를 forwardfill, -999로 결측치 대체
_target = df["target"]
df = df.ffill().fillna(-999).assign(target = _target)

# _type에 따라 train, test 분리
train_df = df.loc[df["_type"]=="train"].drop(columns=["_type"])
test_df = df.loc[df["_type"]=="test"].drop(columns=["_type"])

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [22]:
# data/preprocessed에 저장 없다면 생성
preprocessed_data_path = "data/preprocessed"
os.makedirs(preprocessed_data_path, exist_ok=True)

train_df.to_csv(os.path.join(preprocessed_data_path, "train_df.csv"), index=False)
test_df.to_csv(os.path.join(preprocessed_data_path, "test_df.csv"), index=False)

# 저장 확인
print(f"train_df.csv 파일이 {preprocessed_data_path}에 저장되었습니다.")
print(f"test_df.csv 파일이 {preprocessed_data_path}에 저장되었습니다.")


train_df.csv 파일이 data/preprocessed에 저장되었습니다.
test_df.csv 파일이 data/preprocessed에 저장되었습니다.
